In [54]:
import numpy as np
import pandas as pd
import networkx as nx

import stellargraph
from stellargraph.data import BiasedRandomWalk

# Задание - Предсказание уровня экспресси белка

<img src='https://www.researchgate.net/publication/313504607/figure/fig3/AS:459880453677066@1486655453033/Protein-protein-interaction-PPI-network-of-DEGs-by-STRING-The-interaction-score-was.png'>




<div class="alert alert-info">
<b>Про биологию</b>
    
Экспрессия — процесс, в ходе которого наследственная информация от гена (последовательности нуклеотидов ДНК) преобразуется в функциональный продукт — белок. Уровнем экспрессии называют - количество белка, производящегося в этом процессе. Чем выше экспрессия белка, тем большее количество этого белка появляется в клетках человека. 
    
    

<div class="alert alert-info">    
<b>Важность задачи</b>
    
Существует множество причин необходимости в знании уровня экспресии белка. Например - это позволяет ученым разрабатывать лекарственные средства и оптимизировать их разработку. Теперь вам предстоит побыть в роли биоинформатика и помочь науке!
    
</div>


<div class="alert alert-info">
<b>Про Датасет</b>
    
Датасет представляет собой граф взаимойдествия белков. Где узлы это белки, взаимодействие между белками это ребро. 

Для каждого белка известен уровень его экспрессии. Ниже приведен список ребер `edges`. Информация по экспрессии белков, разбитая на `train` и `test`.
   
    
</div>

In [2]:
#Список ребер графа 

edges = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Otus_HW_protein_expression/main/edges.csv", sep=",") # Подгрузим данные
edges.head()

,node_1,node_2
0,344,50
1,344,153
2,344,532
3,344,679
4,344,986


In [3]:
#Подгрузим тренирочную выборку
train = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Otus_HW_protein_expression/main/train.csv", sep=",") # Подгрузим данные
train.head()

,target,node
0,0.251968,11142
1,0.689541,2243
2,0.678245,15514
3,0.272500,20944
4,0.248888,8721


In [4]:
# Подгрузим отложенную выборку для валидации
test = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Otus_HW_protein_expression/main/test.csv", sep=",")
test.head()

,target,node
0,0.279231,817
1,0.380795,9574
2,0.686527,1607
3,0.303594,4782
4,0.367374,24125


<div class="alert alert-info">
<b>Про Задачу</b>
    
Вам предлагается предсказать экспрессию белков (`target`) по приведенным данным для отложенной выборки. Ответы в отложенной выборке `test` даны вам для самостоятельной валидации.


    
   
    

<div class="alert alert-info">
<b>Замечание и комментарии</b>
    
    

По ряду причин датасет был упрощен так, чтобы выполнялись следующие условия:
* у графа одна компонента связанности. 
* удалены слишком крупные хабы
* плотность связей графа уменьшена
* решить задачу можно классическими ML подходами
    
   

<div class="alert alert-info">
<b>Оценка результатов</b>
    


Оценка точности модели будет оцениваться по метрике MSE на отложенной выборке `test`
        
</div>

<div class="alert alert-info">
<b>Автор задачи</b>

По всем дополнительным вопросами писать Александру Миленькину
* Телеграмм: Alerin75infskin
* Почта: milenkin.aa@phystech.edu
        
</div>

In [6]:
edges.head()

,node_1,node_2
0,344,50
1,344,153
2,344,532
3,344,679
4,344,986


In [8]:
train.head()

,target,node
0,0.251968,11142
1,0.689541,2243
2,0.678245,15514
3,0.272500,20944
4,0.248888,8721


Будем строить Node2Vec представление для узлов в графе.  
Для этого сделаем random walk с помощью библиотеки StellarGraph.  
Далее обучим на корпусе из всех узлов графа и последовательностей random walks векторное представление.

In [27]:
G = nx.from_pandas_edgelist(edges, "node_1", "node_2", create_using=nx.Graph())

In [28]:
G_st = stellargraph.StellarGraph.from_networkx(G)

In [31]:
rw = BiasedRandomWalk(G_st)

In [34]:
walks = rw.run(nodes=list(G_st.nodes()), length=15, n=10, p=0.3)

In [39]:
from gensim.models import Word2Vec

str_walks = [[str(n) for n in walk] for walk in walks]
model = Word2Vec(str_walks, vector_size=30, window=5, min_count=0, sg=1, workers=3, epochs=2)

In [55]:
X_train = np.array([model.wv[str(node)] for node in train['node'].values])
X_test = np.array([model.wv[str(node)] for node in test['node'].values])

In [57]:
y_train = train['target'].values
y_test = test['target'].values

In [58]:
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

Простая модель - будем предсказывать просто среднее

In [68]:
y_pred_train = [y_train.mean() for _ in range(X_train.shape[0])]
y_pred_test = [y_train.mean() for _ in range(X_test.shape[0])]

print(f"MAE of train = {round(mean_absolute_error(y_pred_train, y_train), 2)}, of test = {round(mean_absolute_error(y_pred_test, y_test), 2)}")

MAE of train = 0.33, of test = 0.36


Линейная регерссия

In [60]:
model = LinearRegression().fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

print(f"MAE of train = {round(mean_absolute_error(y_pred_train, y_train), 2)}, of test = {round(mean_absolute_error(y_pred_test, y_test), 2)}")

MAE of train = 0.29, of test = 0.3


Бустинг

In [67]:
model = LGBMRegressor(n_estimators=100, max_depth=10).fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

print(f"MAE of train = {round(mean_absolute_error(y_pred_train, y_train), 2)}, of test = {round(mean_absolute_error(y_pred_test, y_test), 2)}")

MAE of train = 0.11, of test = 0.16
